In [13]:
import pandas as pd

In [14]:
sp = pd.read_csv('../Database/sp500.csv', index_col=0)

In [15]:
index = sp['Adj Close'].pct_change().dropna()

In [16]:

col = ['06/1999-12/1999', '01/2000-12/2002(9/11)', '01/2003-12/2006', '01/2007-12/2009(GFC)',
       '01/2010-12/2019', '01/2020-12/2022(Covid-19)', 'Overall']

period = [range(1, 100), range(100, 285), range(285, 333), range(333, 369), range(369, 380), range(380, 391),
          range(1, 391)]

In [17]:
# index.T.to_csv('../Database/sp500_return.csv')

In [18]:
import os
import numpy as np
import pandas as pd
from PCA_and_ETC import read_and_preprocess_data

# cl_list = ['60_characteristics_us', '70_characteristics_us', '80_characteristics_us', '90_characteristics_us']
cl_list = [f'{i}_characteristics_us' for i in range(10, 80, 10)]
for a in cl_list:
    input_dir = f'../Database/Clustering_Result/Contrastive_Learning2/{a}'
    input_dir2 = f'../Database/Clustering_Result/Contrastive_Learning2/Prob/{a}'
    output_dir = f'../Database/Clustering_Result/Contrastive_Learning2/CL_Pre/{a}'

    files = sorted(filename for filename in os.listdir(input_dir))

    for file in files:
        data = read_and_preprocess_data(input_dir, file)
        prob = read_and_preprocess_data(input_dir2, file)
        prob.index = prob['firms']
        prob = prob.drop(columns='firms')
        prob_filtered = prob[(prob.max(axis=1) <= prob.max(axis=1).quantile(0.70))]
        data.loc[prob_filtered.index, 'clusters'] = 0
        data.to_csv(os.path.join(output_dir, file))